# transclusion of data with jinja2/templ~~at~~es

In [1]:
    import pidgy.base, jinja2.meta, IPython, sys, traitlets

    import pidgy.base, jinja2.meta, IPython, sys, traitlets

In [2]:
    minify = lambda x: __import__('htmlmin').minify(x, False, True, True, True, True, True, True)


    minify = lambda x: __import__('htmlmin').minify(x, False, True, True, True, True, True, True)

In [3]:
    def active_types(shell=None):
        shell = shell or IPython.get_ipython()
        if shell:
            object = list(shell.display_formatter.active_types)
            object.insert(object.index('text/html'), object.pop(object.index('text/latex')))
            return reversed(object)
        return []

    def active_types(shell=None):
        shell = shell or IPython.get_ipython()
        if shell:
            object = list(shell.display_formatter.active_types)
            object.insert(object.index('text/html'), object.pop(object.index('text/latex')))
            return reversed(object)
        return []

In [4]:
    def environment(shell=None):        
        environment = __import__('nbconvert').exporters.TemplateExporter().environment
        environment.loader.loaders.append(jinja2.FileSystemLoader('.'))
        environment.display_manager = DisplayManager()
        environment.finalize = Finalize(parent=shell or IPython.get_ipython())
        return environment

    def environment(shell=None):        
        environment = __import__('nbconvert').exporters.TemplateExporter().environment
        environment.loader.loaders.append(jinja2.FileSystemLoader('.'))
        environment.display_manager = DisplayManager()
        environment.finalize = Finalize(parent=shell or IPython.get_ipython())
        return environment

In [5]:
    class MarkdownDisplay(pidgy.base.Trait):
        body = traitlets.Unicode()
        
        vars = traitlets.Set()
        template = traitlets.Any()
        _display = traitlets.Any()
        @traitlets.default('template')
        def _default_template(self):
            return self.environment.from_string(self.body)
        @traitlets.default('vars')
        def _default_vars(self):
            return jinja2.meta.find_undeclared_variables(self.template.environment.parse(self.body))
            
        def update(self, **kwargs):
            if self._display is None: return self.display(**kwargs)
            md = IPython.display.Markdown(self.template.render(**kwargs))
            self._display.update(md)
        
        @property
        def render(self,): return self.template.render
        
        def display(self, **kwargs):
            md = IPython.display.Markdown(self.render(**kwargs))
            if self.vars:
                if self._display is None:
                    self._display = IPython.display.display(md, display_id=True)
                else:
                    self._display.display(md)
            else:
                IPython.display.display(md)
                
        _ipython_display_ = display

    class MarkdownDisplay(pidgy.base.Trait):
        body = traitlets.Unicode()
        
        vars = traitlets.Set()
        template = traitlets.Any()
        _display = traitlets.Any()
        @traitlets.default('template')
        def _default_template(self):
            return self.environment.from_string(self.body)
        @traitlets.default('vars')
        def _default_vars(self):
            return jinja2.meta.find_undeclared_variables(self.template.environment.parse(self.body))
            
        def update(self, **kwargs):
            if self._display is None: return self.display(**kwargs)
            md = IPython.display.Markdown(self.template.render(**kwargs))
            self._display.update(md)
        
        @property
        def render(self,): return self.template.render
        
        def display(self, **kwargs):
            md = IPython.display.Markdown(self.render(**kwargs))
            if self.vars:
                if self._display is None:
                    self._display = IPython.display.display(md, display_id=True)
                else:
                    self._display.display(md)
            else:
                IPython.display.display(md)
                
        _ipython_display_ = display

In [6]:
    class DisplayManager(pidgy.base.Trait):
        display = traitlets.Dict()
        state = traitlets.Dict()
        widgets = traitlets.List()
        def pre_execute(self):
            deleted = getattr(self.parent, '_last_parent', {}).get('metadata', {}).get('deletedCells', [])
            for key, displays in self.display.items() if deleted else []:
                self.display[key] = [
                    x for x in displays if x._display and x._display.display_id not in deleted
                ]
                
        def append(self, object):
            for key in object.vars:
                self.display[key] = self.display.get(key, [])
                self.display[key].append(object)
                self.state[key] = self.parent.user_ns.get(key, None)
        
        def post_execute(self):
            update = {
                x: self.parent.user_ns.get(x, None) for x in self.display 
                if isinteractive(self.parent.user_ns.get(x, None)) or
                self.parent.user_ns.get(x, None) is not self.state.get(x, None)
            }
            for key, object in update.items():
                if isinteractive(object) and object not in self.widgets:
                    object.observe(lambda x: self.post_execute())
                    self.widgets += [object]
            self.state.update(update)
            for object in set(
                sum([self.display[x] for x in update], [])
            ): object.update(**self.state)

    class DisplayManager(pidgy.base.Trait):
        display = traitlets.Dict()
        state = traitlets.Dict()
        widgets = traitlets.List()
        def pre_execute(self):
            deleted = getattr(self.parent, '_last_parent', {}).get('metadata', {}).get('deletedCells', [])
            for key, displays in self.display.items() if deleted else []:
                self.display[key] = [
                    x for x in displays if x._display and x._display.display_id not in deleted
                ]
                
        def append(self, object):
            for key in object.vars:
                self.display[key] = self.display.get(key, [])
                self.display[key].append(object)
                self.state[key] = self.parent.user_ns.get(key, None)
        
        def post_execute(self):
            update = {
                x: self.parent.user_ns.get(x, None) for x in self.display 
                if isinteractive(self.parent.user_ns.get(x, None)) or
                self.parent.user_ns.get(x, None) is not self.state.get(x, None)
            }
            for key, object in update.items():
                if isinteractive(object) and object not in self.widgets:
                    object.observe(lambda x: self.post_execute(), 'value')
                    self.widgets += [object]
            self.state.update(update)
            for object in set(
                sum([self.display[x] for x in update], [])
            ): object.update(**self.state)

In [7]:
    class Finalize(pidgy.base.Trait):
        def normalize(self, key, object, metadata):            
            if key.startswith('image'):
                if 'svg' in key: return minify(object)
                width, height = metadata.get(key, {}).get('width'), metadata.get(key, {}).get('height')
                if isinstance(object, bytes):
                    object = __import__('base64').b64encode(object).decode('utf-8')
                object = F"""<img src="data:image/{key.partition('/')[2]};base64,{object}"/>"""
            if key == 'text/html': object = minify(object)
            return object
        
        def __call__(self, object):
            datum = self.parent.display_formatter.format(object)
            data, metadata = datum if isinstance(datum, tuple) else (datum, {})
            try: key = next(filter(data.__contains__, active_types(self.parent)))
            except StopIteration: return str(object)
            if key == 'text/plain': return str(object)
            return self.normalize(key, data[key], metadata)

    class Finalize(pidgy.base.Trait):
        def normalize(self, key, object, metadata):            
            if key.startswith('image'):
                if 'svg' in key: return minify(object)
                width, height = metadata.get(key, {}).get('width'), metadata.get(key, {}).get('height')
                if isinstance(object, bytes):
                    object = __import__('base64').b64encode(object).decode('utf-8')
                object = F"""<img src="data:image/{key.partition('/')[2]};base64,{object}"/>"""
            if key == 'text/html': object = minify(object)
            return object
        
        def __call__(self, object):
            datum = self.parent.display_formatter.format(object)
            data, metadata = datum if isinstance(datum, tuple) else (datum, {})
            try: key = next(filter(data.__contains__, active_types(self.parent)))
            except StopIteration: return str(object)
            if key == 'text/plain': return str(object)
            return self.normalize(key, data[key], metadata)

In [8]:
    def iswidget(object):
        if 'ipywidgets' in sys.modules:
            if isinstance(object, __import__('ipywidgets').Widget):
                return True
        return False

    def ispanel(object):
        if 'param' in sys.modules:
            if isinstance(object, __import__('param').Parameterized):
                return True
        return False

    def isinteractive(object):
        return iswidget(object) | ispanel(object)

    def iswidget(object):
        if 'ipywidgets' in sys.modules:
            if isinstance(object, __import__('ipywidgets').Widget):
                return True
        return False

    def ispanel(object):
        if 'param' in sys.modules:
            if isinstance(object, __import__('param').Parameterized):
                return True
        return False

    def isinteractive(object):
        return iswidget(object) | ispanel(object)